<a href="https://colab.research.google.com/github/Tarasila/Mask_RCNN_defect_detection/blob/master/Mask_RCNN_defect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# download files to Google Drive
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

weights_file = drive.CreateFile({'title' : 'weights_16_05.h5'})
weights_file.SetContentFile('/logs/dimple20190516T0902/mask_rcnn_dimple_0005.h5')
weights_file.Upload()

drive.CreateFile({'id': weights_file.get('id')})


GoogleDriveFile({'id': '1jT7HIhw8IV2o0KHkblgfwhFYcG8rQ9go'})

In [0]:
import os
import shutil

from google.colab import drive
drive.mount('/content/gdrive')

Mask_RCNN_folder = os.path.join("/content/gdrive/My Drive/", 'Mask_RCNN_dimple')

GitHub essentials

In [0]:
# copy essential folders to local environment
# - dataset
# - weights
!cp -r '/content/gdrive/My Drive/dataset' dataset
!cp -r '/content/gdrive/My Drive/weights' weights


In [0]:
# copy 'Mask_RCNN_dimple' folder to local environment
!cp -r '/content/gdrive/My Drive/Mask_RCNN_dimple' Mask_RCNN

In [0]:
# install all requirements
os.chdir('Mask_RCNN')

!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

In [0]:
#os.chdir(os.pardir)
#!ls
!python dimple.py train --dataset=dataset/ --weights=/logs/dimple20190516T0902/mask_rcnn_dimple_0005.h5

Traceback (most recent call last):
  File "dimple.py", line 456, in <module>
    train(model)
  File "dimple.py", line 291, in train
    layers='heads')
  File "/content/Mask_RCNN/mrcnn/model.py", line 2374, in train
    use_multiprocessing=True,
  File "/usr/local/lib/python3.6/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1418, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py", line 262, in fit_generator
    val_enqueuer.stop()
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 496, in stop
    self.run_thread.join(timeout)
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrup

In [0]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import dimple

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

custom_WEIGHTS_PATH = sorted(glob.glob("/logs/*/mask_rcnn_*.h5"))[-1]
#custom_WEIGHTS_PATH = '/weights/weights_15_05.h5'

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

config = dimple.DimpleConfig()
custom_DIR = os.path.join(ROOT_DIR, "dataset")

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax
  
# Load validation dataset
dataset = dimple.DimpleDataset()
dataset.load_dimple(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)

In [0]:
custom_WEIGHTS_PATH = sorted(glob.glob("/logs/*/mask_rcnn_*.h5"))[-1]
print(custom_WEIGHTS_PATH)

/logs/dimple20190516T0902/mask_rcnn_dimple_0005.h5


In [0]:
#image_id = random.choice(dataset.image_ids)
for image_id in dataset.image_ids:
  image, image_meta, gt_class_id, gt_bbox, gt_mask =\
      modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
  info = dataset.image_info[image_id]
  print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                         dataset.image_reference(image_id)))

  # Run object detection
  results = model.detect([image], verbose=1)

  # Display results
  ax = get_ax(1)
  r = results[0]
  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              dataset.class_names, r['scores'], ax=ax,
                              title="Predictions")
  log("gt_class_id", gt_class_id)
  log("gt_bbox", gt_bbox)
  log("gt_mask", gt_mask)